In [1]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

import peyes.Config.constants as cnst
import peyes.Config.experiment_config as cnfg

from peyes.DataSetLoaders.Lund2013DataSetLoader import Lund2013DataSetLoader as Lund2013

from peyes.GazeDetectors.IVTDetector import IVTDetector
from peyes.GazeDetectors.IDTDetector import IDTDetector
from peyes.GazeDetectors.EngbertDetector import EngbertDetector
from peyes.GazeDetectors.NHDetector import NHDetector
from peyes.GazeDetectors.REMoDNaVDetector import REMoDNaVDetector

import peyes.Visualization.scarfplot as scarfplot

pio.renderers.default = 'notebook'

## Load Data Set

In [2]:
lund_dataset = Lund2013().load(should_save=False)

In [3]:
trial1 = lund_dataset[lund_dataset[cnst.TRIAL] == 2]
t, x, y = trial1[cnst.T].to_numpy(), trial1[cnst.X].to_numpy(), trial1[cnst.Y].to_numpy()
mn_events = trial1["MN"].to_numpy()
ra_events = trial1["RA"].to_numpy()

trial1

## Detect Events

In [4]:
%%capture --no-stdout

ivt = IVTDetector()
ivt_results = ivt.detect(t, x, y)

idt = IDTDetector()
idt_results = idt.detect(t, x, y)

engbert = EngbertDetector()
engbert_results = engbert.detect(t, x, y)

nh = NHDetector()
nh_results = nh.detect(t, x, y)

rmdnv = REMoDNaVDetector()
rmdnv_results = rmdnv.detect(t, x, y)

### Figure 1: Gaze Locations
Scatter plot of X and Y coordinates, colored by each sample's event (pre-annotated) 

In [5]:
colors = np.array([cnfg.EVENT_MAPPING[val]["color"] for val in mn_events])

# TODO: add background image

fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=x, y=y, mode='markers', marker=dict(color=colors, size=5)))
fig1.update_layout(xaxis_title="X", yaxis_title="Y")
fig1.show()

### Figure 2: Gaze over Time
Presenting the X- and Y-coordinates over time, with the velocity and/or acceleration (normalized to avoid scale issues).
Bottom of the figure will present the detected events.

In [6]:
# normalize velocity & acceleration
v, a = nh_results[cnst.GAZE][cnst.VELOCITY], nh_results[cnst.GAZE][cnst.ACCELERATION]
nv = v / np.nanmax(v)
na = a / np.nanmax(a)

# create the figure with two y-axes
fig2 = make_subplots(specs=[[{"secondary_y": True}]])

# add lines for gaze location & velocity
fig2.add_trace(go.Scatter(x=t, y=x, mode="lines", line=dict(color='#ff0000', width=4), name="X"), secondary_y=False)
fig2.add_trace(go.Scatter(x=t, y=y, mode="lines", line=dict(color='#0000ff', width=4), name="Y"), secondary_y=False)
fig2.add_trace(go.Scatter(x=t, y=nv, mode="lines", line=dict(color='#888888', width=2, dash='dash'), name="v"), secondary_y=True)
# fig2.add_trace(go.Scatter(x=t, y=na, mode="lines", line=dict(color='#888888', width=2, dash='dot'), name="a"), secondary_y=True)

# add events as heatmap
fig2 = scarfplot.add_scarfplot(fig2, t, mn_events, 0, 0.5 * np.nanmin([x, y]))

# move legend to top left
fig2.update_layout(xaxis_title="Time (ms)",
                   yaxis_title="Gaze Location",
                   yaxis2_title="Velocity",
                   legend=dict(
                       yanchor="top",
                       y=0.99,
                       xanchor="left",
                       x=0.01
                   ))

fig2.show()

### Figure 3: Gaze over Time with Multiple Detectors
Adding the detection results for NHDetector and EngbertDetector to the previous figure.
Also, playing around with some layout properties like the background colors, text colors, etc.

In [7]:
nh_events = nh_results[cnst.GAZE][cnst.EVENT]
eng_events = engbert_results[cnst.GAZE][cnst.EVENT]

fig3 = go.Figure(fig2)
line_size = 0.5 * np.nanmin([x, y])

fig3 = scarfplot.add_scarfplot(fig3, t, nh_events, -2 * line_size, -1 * line_size)
fig3 = scarfplot.add_scarfplot(fig3, t, eng_events, -4 * line_size, -3 * line_size)

fig3.update_layout(
    plot_bgcolor='white',
    paper_bgcolor='white',
    font_color='black',
    xaxis=dict(showgrid=False),
    yaxis1=dict(showgrid=False),
    yaxis2=dict(showgrid=False),
)

fig3.show()

### Figure 4: Comparing Detectors
Displaying the detected events for each detector in a separate (heatmap) subplot depicting events over time.

In [8]:
ivt_events = ivt_results[cnst.GAZE][cnst.EVENT]
idt_events = idt_results[cnst.GAZE][cnst.EVENT]
nh_events = nh_results[cnst.GAZE][cnst.EVENT]
eng_events = engbert_results[cnst.GAZE][cnst.EVENT]
rmdnv_events = rmdnv_results[cnst.GAZE][cnst.EVENT]

events_dict = {
    "IVT": ivt_events,
    "IDT": idt_events,
    "Engbert": eng_events,
    "NH": nh_events,
    "REMoDNaV": rmdnv_events,
    "MN": mn_events,
    "RA": ra_events,
}

ROW_WIDTH = 10

fig4 = go.Figure()
for i, (_, events) in enumerate(events_dict.items()):
    fig4 = scarfplot.add_scarfplot(fig4, t, events, ymin=2 * i * ROW_WIDTH, ymax=(2 * i + 1) * ROW_WIDTH)

fig4.update_layout(yaxis=dict(range=[0, (2 * len(events_dict) - 1) * ROW_WIDTH],
                              tickmode='array',
                              tickvals=[(2 * i + 0.5) * ROW_WIDTH for i in range(len(events_dict))],
                              ticktext=list(events_dict.keys())),
                   xaxis_title="Time (ms)", )


fig4.show()

In [9]:
# TODO:
# add background image to the first figure
# create a fixation/gaze heatmap figure
# create a main-sequence plot